In [1]:
from random import random
from random import uniform
import math
import skimage.measure
import cv2
import numpy as np
import matlab.engine
import matplotlib.pyplot as plt

In [2]:
eng = matlab.engine.start_matlab()

In [3]:
def gTransformation(img,a,b,c,k):
# g(x,y)
# M = No of rows
# N = No of columns
# img as numpy array
  #a = 2
  #b = 5
  #c = 5
  #k = 1
  M = np.size(img,0)
  N = np.size(img,1)
  n = int(M/20)
  g = np.zeros((M,N))
  gMean,S = cv2.meanStdDev(img)
  for i in range(M):
    for j in range(N):

      xstart = i-n
      xend = i+n
      ystart = j-n
      yend = j+n

      if(xstart<0):
        xstart = i
      if(xend>=N):
        xend = i
      if(ystart<0):
        ystart = j
      if(yend>=M):
        yend = j
      mean, sigma = cv2.meanStdDev(img[xstart:xend, ystart:yend])
      #print(mean)
      #print((k*gMean)/(sigma + b)*(img[i,j] - c*mean) + mean**a)
      g[i,j] = (k*gMean)/(sigma + b)*(img[i,j] - c*mean) + mean**a

      #print(max_img)
      #print(min_img)
      
  max_img = np.max(g)
  min_img = np.min(g)     
  for i in range(M):
    for j in range(N):
      g[i][j]=g[i][j]*256/(max_img-min_img);
      g[i][j] = int(g[i][j])
        
  
  return g
            

In [4]:
def sobel_custom(grayscale_image):
  Gx = np.array([[1.0, 0.0, -1.0], [2.0, 0.0, -2.0], [1.0, 0.0, -1.0]])
  Gy = np.array([[1.0, 2.0, 1.0], [0.0, 0.0, 0.0], [-1.0, -2.0, -1.0]])
  [rows, columns] = np.shape(grayscale_image)  # we need to know the shape of the input grayscale image
  sobel_filtered_image = np.zeros(shape=(rows, columns))  # initialization of the output image array (all elements are 0)
  for i in range(rows - 2):
    for j in range(columns - 2):
        gx = np.sum(np.multiply(Gx, grayscale_image[i:i + 3, j:j + 3]))  # x direction
        gy = np.sum(np.multiply(Gy, grayscale_image[i:i + 3, j:j + 3]))  # y direction
        sobel_filtered_image[i + 1, j + 1] = np.sqrt(gx ** 2 + gy ** 2)  # calculate the "hypotenuse"
  sobel_filtered_image = sobel_filtered_image*256
  return sobel_filtered_image

In [5]:
def fitnessFunc(img,x):
  sum = 0;
  k = 0;
  a = x[0]
  b = x[1]
  c = x[2]
  k = x[3]
  Gimg = gTransformation(img,a,b,c,k)
  sobel_img = sobel_custom(Gimg)
  M = sobel_img.shape[0]
  N = sobel_img.shape[1]
  for i in range(sobel_img.shape[0]):
    for j in range(sobel_img.shape[1]):
      if(img[i][j]>50):
        sum = sum+sobel_img[i][j];
    

  for i in range(sobel_img.shape[0]):
    for j in range(sobel_img.shape[1]):
      if(img[i][j]>180):
        k = k+1;
  E = sum//k
  Y = math.log(E)
  Z = math.log(Y)
  J = eng.edge(Gimg,'Sobel')
    #[jj,pp] = eng.bwlabel(J)
  cc = eng.bwconncomp(J)
  cedge = cc['NumObjects']
  no_of_edges = cedge #from matlab

  H = skimage.measure.shannon_entropy(img)
  s = M*N;
  T = no_of_edges/s
  F = Z*H*T

  return 1/F

In [6]:
#--- MAIN ---------------------------------------------------------------------+

class Particle:
    def __init__(self, x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(img,self.position_i)

        # check to see if the current position is an individual best
        if self.err_i<self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i.copy()
            self.err_best_i=self.err_i
                    
    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant
        
        for i in range(0,num_dimensions):
            r1=random()
            r2=random()
            
            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]
            
            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i]<bounds[i][0]:
                self.position_i[i]=bounds[i][0]
        
        
def minimize(costFunc, x0, bounds, num_particles, maxiter, verbose=False):
    global num_dimensions

    num_dimensions=len(x0)
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group

    # establish the swarm
    swarm=[]
    for i in range(0,num_particles):
        swarm.append(Particle(x0))

    # begin optimization loop
    i=0
    while i<maxiter:
        if verbose: 
            print(f'iter: {i:>4d}, best solution: {err_best_g:10.6f}')
            print(pos_best_g)
            #currentimg = gTransformation(img,pos_best_g[0],pos_best_g[1],pos_best_g[2],pos_best_g[3])
            #plt.imshow(currentimg,cmap='gray')
            
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j].evaluate(costFunc)

            # determine if current particle is the best (globally)
            if swarm[j].err_i<err_best_g or err_best_g==-1:
                pos_best_g=list(swarm[j].position_i)
                err_best_g=float(swarm[j].err_i)
        
        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j].update_velocity(pos_best_g)
            swarm[j].update_position(bounds)
        i+=1

    # print final results
    if verbose:
        print('\nFINAL SOLUTION:')
        print(f'   > {pos_best_g}')
        print(f'   > {err_best_g}\n')

    return err_best_g, pos_best_g

In [7]:
img = cv2.imread("cameraman.png")
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#f = fitnessFunc(gray_image)
#f
img = gray_image

In [8]:
initial=[1.3884230563154893, 0.15466137373411903, 0.9274511132456694, 1.276719804958083]               # initial starting location [x1,x2...]
bounds=[(-10,10),(-10,10),(-10,10),(0.5,1.5)]

In [9]:
minimize(fitnessFunc, initial, bounds, num_particles=15, maxiter=30, verbose=True)

iter:    0, best solution:  -1.000000
[]
iter:    1, best solution:  37.787775
[2, 0.5, 1.5, 1.4]


KeyboardInterrupt: 

In [ ]:
aopt = 1.3828390678652576
bopt = -0.051058268158485805
copt = 1.0549935426028143
kopt = 0.5
enhancedimg = gTransformation(img,aopt,bopt,copt,kopt)
plt.imshow(enhancedimg,cmap='gray')

In [ ]:
enhancedimg